# En este notebook se encuentras las funciones utilizadas a lo largo del proyecto
<span style="font-size:Large;">       
Se ha instalado el siguiente paquete para cargar las funciones entre notebooks:<br>    
<br> 
    
```python
! pip install nbimporter
```
<br>     
</span>

In [1]:
# Básicos:
import matplotlib.pyplot as plt
from collections import Counter
from joblib import dump, load
import tensorflow as tf
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import keras
import glob
import cv2
import os

# CNN:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split

# Métricas:
from sklearn.metrics import confusion_matrix

# Pretrained CNN:
from tensorflow.keras.applications import Xception

# Time:
import tqdm

# Funciones procesamiento de imagenes

In [2]:
def image_processor(images, high, width):
    images_processed = []

    for image in images:
        image_processed = image / 255
        image_resized = cv2.resize(src = image_processed, dsize = (high, width))
        images_processed.append(image_resized)

    return images_processed

In [3]:
def augment_data(X_train, y_train):

    data_augmentation = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                             layers.experimental.preprocessing.RandomRotation(0.2),
                                             layers.experimental.preprocessing.RandomZoom(0.1)])

    X_train_aug = []
    y_train_aug = []

    for image, breed in tqdm.tqdm(zip(X_train, y_train)):

        img = tf.expand_dims(image, 0)
        for i in range(9):

            augmented_image = data_augmentation(img)
            X_train_aug.append(augmented_image[0])
            y_train_aug.append(breed)
            
    return X_train_aug, y_train_aug

# Funciones Red Neuronal

- ## Custom CNN

In [4]:
def custom_cnn(X_train, y_train, X_val, y_val, epochs, batch, call_backs= True):
    
    num_classes = len(y_train[0])

    model = Sequential()

    # Feature training:
        # Capa de entrada:
    model.add(Conv2D(filters= 16, kernel_size= 2, padding= 'same', activation= 'relu', input_shape= (224, 224, 3)))
    model.add(MaxPooling2D(pool_size= 2))

        # Conv_pooling_1
    model.add(Conv2D(filters= 32, kernel_size= 2 , padding= 'same' , activation= 'relu'))
    model.add(MaxPooling2D(pool_size= 2))

        # Conv_pooling_2
    model.add(Conv2D(filters= 64 , kernel_size= 2 , padding= 'same' , activation= 'relu'))
    model.add(MaxPooling2D(pool_size= 2))
    model.add(Dropout(0.4))

        # Conv_pooling_3
    model.add(Conv2D(filters= 128 , kernel_size= 2 , padding= 'same' , activation= 'relu'))
    model.add(MaxPooling2D(pool_size= 2))
    model.add(Dropout(0.4))

    model.add(Flatten())

    # Fully conecting
    model.add(Dense(512, activation= 'relu'))
    model.add(Dropout(0.4))

    # Salida:
    model.add(Dense(num_classes, activation= 'softmax'))
    model.summary()
    
    model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])
    
    if call_backs:
        # Callbacks:
        checkpoint = ModelCheckpoint('best_model_5_1.h5', monitor= 'val_loss', verbose= 0, save_best_only= True)
        early_stops = EarlyStopping(patience= 4, monitor= 'val_loss')

        # Entrenamos el modelo:
        history = model.fit(x= X_train, y= y_train,
                            validation_data = (X_val, y_val),
                            batch_size= batch,
                            epochs= epochs,
                            callbacks= [checkpoint, early_stops]
                           )
    else:
        history = model.fit(x= X_train, y= y_train,
                            validation_data = (X_val, y_val),
                            batch_size= batch,
                            epochs= epochs)
        
    
    return history

- ## Xception

In [5]:
def xception_cnn(X_train, y_train, X_val, y_val, epochs, batch, call_backs= True):
    
    num_classes = len(y_train[0])
    shape = X_train.shape[1:]
    
    xception_model = Xception(weights = "imagenet", include_top = False, input_shape = shape)
    
    model = Sequential()

    model.add(xception_model)

    model.add(Flatten())

    # Añado una fully conecting layer para crear nuevas conexiones, ajustandose a nuestros datos:
    model.add(Dense(512, activation= 'relu'))
    model.add(Dropout(0.4))

    # Salida:
    model.add(Dense(num_classes, activation= 'softmax'))
    model.summary()
    
    model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])
    
    if call_backs:
        # Callbacks:
        checkpoint = ModelCheckpoint('best_model_5_2.h5', monitor= 'val_loss', verbose= 0, save_best_only= True)
        early_stops = EarlyStopping(patience= 4, monitor= 'val_loss')

        # Entrenamos el modelo:
        history = model.fit(x= X_train, y = y_train,
                            validation_data = (X_val, y_val),
                            batch_size= batch,
                            epochs= epochs,
                            callbacks= [checkpoint, early_stops]
                           )
    else:
        history = model.fit(x= X_train, y= y_train,
                            validation_data = (X_val, y_val),
                            batch_size= batch,
                            epochs= epochs)
    
    
    return history

# Visualización resultados Red Neuronal

In [6]:
def plot_history(history):
    
    accuracy     = history.history["accuracy"]
    loss         = history.history["loss"]

    val_accuracy = history.history["val_accuracy"]
    val_loss     = history.history["val_loss"]

    epochs = range(1, len(accuracy) + 1)
    
    # Plots:    
    figure, axes = plt.subplots(1, 2, figsize = (12, 6))
    axes = axes.flatten() 

    # Plot Accuracy
    axes[0].plot(epochs, accuracy, "r--", label="Train accuracy")
    axes[0].plot(epochs, val_accuracy, "b", label="Validation accuracy")

    axes[0].set_title("Training and validation accuracy")
    axes[0].set_ylabel("Accuracy")
    axes[0].set_xlabel("Epochs")
    axes[0].legend()

    # Plot Loss
    axes[1].plot(epochs, loss, "r--", label="Train loss")
    axes[1].plot(epochs, val_loss, "b", label="Validation loss")

    axes[1].set_title("Training and validation loss")
    axes[1].set_ylabel("Loss")
    axes[1].set_xlabel("Epochs")
    axes[1].legend()

    plt.show()